In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING", "true")
HF_TOKEN = os.getenv("HF_TOKEN")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain_groq import ChatGroq

# model = ChatGroq(model="Gemma2-9b-It", groq_api_key=GROQ_API_KEY)
model = ChatGroq(model="Gemma2-9b-It")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke(
    [HumanMessage(content="Hi , My name is YASH and I am a Chief AI Engineer")]
)

AIMessage(content="Hello YASH! It's nice to meet you. \n\nThat's a very impressive title! As a large language model, I'm always interested in learning more about how people are using AI in innovative ways. \n\nWhat kind of projects are you working on as Chief AI Engineer?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 23, 'total_tokens': 87, 'completion_time': 0.116363636, 'prompt_time': 0.002191188, 'queue_time': 0.241696911, 'total_time': 0.118554824}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c9745c0-8002-4b51-bc95-e40ed82c7bfe-0', usage_metadata={'input_tokens': 23, 'output_tokens': 64, 'total_tokens': 87})

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi , My name is YASH and I am a Chief AI Engineer"),
        AIMessage(
            content="Hello YASH! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"
        ),
        HumanMessage(content="Hey What's my name and what do I do?"),
    ]
)

AIMessage(content="You are YASH, and you are a Chief AI Engineer!  😄 \n\nIs there anything else you'd like me to remember about you?  Perhaps your favorite AI project or a fun fact about your work?  The more I know, the better I can interact with you. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 99, 'total_tokens': 162, 'completion_time': 0.114545455, 'prompt_time': 0.005849677, 'queue_time': 0.24735668300000002, 'total_time': 0.120395132}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--caf900cf-1ff0-43f9-b322-af76de96283e-0', usage_metadata={'input_tokens': 99, 'output_tokens': 63, 'total_tokens': 162})

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi , My name is YASH and I am a Chief AI Engineer")],
    config=config,
)

In [8]:
response.content

"Hello YASH, it's nice to meet you! It's impressive to hear that you're a Chief AI Engineer.  \n\nWhat kind of AI projects are you working on these days? I'm always interested to learn about the innovative work being done in the field.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is YASH, as you told me earlier. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 98, 'total_tokens': 126, 'completion_time': 0.050909091, 'prompt_time': 0.006617926, 'queue_time': 0.242822373, 'total_time': 0.057527017}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c5fc301f-e3ad-4ad8-9f8c-770315938bda-0', usage_metadata={'input_tokens': 98, 'output_tokens': 28, 'total_tokens': 126})

In [10]:
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")], config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")], config=config1
)
response.content

'Hey John! Nice to meet you. 👋\n\nHow can I help you today?  \n\n'

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")], config=config1
)
response.content

'Your name is John, remember? 😊 \n\nI try my best to keep track of what we talk about!  \n\n\n\n\n'

### Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.


In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the question to the best of your ability",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [14]:
chain.invoke({"messages": [HumanMessage(content="Hi My name is YASH")]})

AIMessage(content="Hi YASH, it's nice to meet you!  \n\nI'm ready to answer your questions to the best of my ability. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 31, 'total_tokens': 69, 'completion_time': 0.069090909, 'prompt_time': 0.002278569, 'queue_time': 0.24556563099999998, 'total_time': 0.071369478}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7565774d-acd4-40a7-9513-fd564c4b7ee1-0', usage_metadata={'input_tokens': 31, 'output_tokens': 38, 'total_tokens': 69})

In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is YASH")], config=config
)

response

AIMessage(content="Hello YASH! It's nice to meet you. \n\nI'm ready to help with any questions you have. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 31, 'total_tokens': 67, 'completion_time': 0.065454545, 'prompt_time': 0.002285179, 'queue_time': 0.24556411, 'total_time': 0.067739724}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d4b3a2c6-97ee-4eba-b1f5-9d2892dd473a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 36, 'total_tokens': 67})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is YASH.  \n\nI remember that you told me! 😊  \n\nIs there anything else I can help you with?\n'

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response = chain.invoke(
    {"messages": [HumanMessage(content="Hi My name is YASH")], "language": "Hindi"}
)
response.content

'नमस्ते यश! \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं।  क्या मैं आपकी कोई मदद कर सकता हूँ? \n'

In [20]:
with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I am YASH")], "language": "Hindi"},
    config=config,
)
repsonse.content

'नमस्ते यश! 😊 \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप जो भी पूछना चाहें, मैं अपने ज्ञान के अनुसार उत्तर देने की पूरी कोशिश करूँगा।  \n\n'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

In [23]:
response.content

'Tu nombre es YASH. 😊 \n'

### Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages


In [24]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    # max_tokens=30,
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="5"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='5', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [25]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do i like")],
        "language": "English",
    }
)
response.content

'You like vanilla ice cream!  🍦 😊 \n\n\n\nIs there anything else I can help you with?\n'

In [26]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="whats 2 + 2")],
        "language": "English",
    }
)
response.content

"That's a tricky one!  2 + 2 = 4 😊 \n"

In [27]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id": "chat5"}}

In [28]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob!  😄  \n'

In [29]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?, also the answer which is given")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked:  "Whats 2 + 2" \n\nAnd I answered: "5"  \n\n\n\nLet me know if you have any other questions! 😊 \n'